In [ ]:
import pandas as pd
import re
import numpy as np
import warnings
from scipy.stats import zscore
import time
import random
import undetected_chromedriver as uc
from fake_useragent import UserAgent
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import os
from IPython.display import clear_output, display
from bs4 import BeautifulSoup
import subprocess
import sys

try:
    import matplotlib.pyplot as plt
    import seaborn as sns
    PLOTTING_AVAILABLE = True
except:
    print("Matplotlib não disponível. Os gráficos não serão gerados.")
    PLOTTING_AVAILABLE = False

warnings.filterwarnings("ignore")

def verificar_chrome_instalado():
    """Verifica se o Chrome está instalado no sistema"""
    try:
        # Tentar diferentes caminhos do Chrome no Windows
        chrome_paths = [
            r"C:\Program Files\Google\Chrome\Application\chrome.exe",
            r"C:\Program Files (x86)\Google\Chrome\Application\chrome.exe",
            r"C:\Users\{}\AppData\Local\Google\Chrome\Application\chrome.exe".format(os.getenv('USERNAME', ''))
        ]
        
        for path in chrome_paths:
            if os.path.exists(path):
                print(f"Chrome encontrado em: {path}")
                return True
        
        print("Chrome não encontrado. Por favor, instale o Google Chrome.")
        return False
    except Exception as e:
        print(f"Erro ao verificar Chrome: {e}")
        return False

def atualizar_chromedriver():
    """Tenta atualizar o ChromeDriver automaticamente"""
    try:
        print("Tentando atualizar ChromeDriver...")
        subprocess.run([sys.executable, "-m", "pip", "install", "--upgrade", "undetected-chromedriver"], 
                      check=True, capture_output=True)
        print("ChromeDriver atualizado com sucesso!")
        return True
    except Exception as e:
        print(f"Erro ao atualizar ChromeDriver: {e}")
        return False

def configure_driver():
    try:
        # Verificar se o Chrome está instalado
        if not verificar_chrome_instalado():
            return None
        
        options = uc.ChromeOptions()
        ua = UserAgent()
        user_agent = ua.random
        options.add_argument(f'user-agent={user_agent}')
        options.add_argument('--disable-gpu')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        options.add_argument('--disable-notifications')
        options.add_argument('--disable-popup-blocking')
        options.add_argument('--disable-blink-features=AutomationControlled')
        options.add_argument('--disable-web-security')
        options.add_argument('--disable-features=VizDisplayCompositor')
        options.add_argument('--remote-debugging-port=9222')
        options.add_argument('--disable-extensions')
        options.add_argument('--disable-plugins')
        options.add_argument('--disable-images')
        options.add_argument('--disable-javascript')
        options.add_experimental_option("excludeSwitches", ["enable-automation"])
        options.add_experimental_option('useAutomationExtension', False)
        
        # Configurações adicionais para evitar crashes
        options.add_argument('--disable-background-timer-throttling')
        options.add_argument('--disable-backgrounding-occluded-windows')
        options.add_argument('--disable-renderer-backgrounding')
        options.add_argument('--disable-features=TranslateUI')
        options.add_argument('--disable-ipc-flooding-protection')
        
        # Tentar diferentes versões do ChromeDriver
        driver = None
        tentativas = [
            ("versão automática", lambda: uc.Chrome(options=options, version_main=None)),
            ("versão 120", lambda: uc.Chrome(options=options, version_main=120)),
            ("versão 119", lambda: uc.Chrome(options=options, version_main=119)),
            ("versão 118", lambda: uc.Chrome(options=options, version_main=118)),
            ("versão padrão", lambda: uc.Chrome(options=options))
        ]
        
        for nome_tentativa, tentativa_func in tentativas:
            try:
                print(f"Tentando {nome_tentativa}...")
                driver = tentativa_func()
                print(f"Sucesso com {nome_tentativa}!")
                break
            except Exception as e:
                print(f"Falha com {nome_tentativa}: {str(e)[:100]}...")
                continue
        
        if driver:
            # Configurar timeout e outras propriedades
            driver.set_page_load_timeout(30)
            driver.implicitly_wait(10)
            
            width = random.randint(1050, 1200)
            height = random.randint(800, 960)
            driver.set_window_size(width, height)
            
            # Executar script para ocultar automação
            driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
            
            return driver
        else:
            print("Não foi possível criar o driver em nenhuma tentativa")
            print("Tentando atualizar ChromeDriver...")
            if atualizar_chromedriver():
                print("Tente executar o código novamente após a atualização.")
            return None
            
    except Exception as e:
        print(f"Erro ao configurar o driver: {e}")
        return None

def add_random_actions(driver):
    try:
        driver.execute_script("""
            var event = new MouseEvent('mousemove', {
                'view': window,
                'bubbles': true,
                'cancelable': true,
                'clientX': arguments[0],
                'clientY': arguments[1]
            });
            document.dispatchEvent(event);
        """, random.randint(0, 800), random.randint(0, 600))
        time.sleep(random.uniform(0.5, 1.5))
    except Exception as e:
        print(f"Erro nas ações aleatórias: {e}")

def verificar_carregamento_completo(driver, timeout=15):
    print("\nVerificando carregamento completo...")
    last_count = 0
    stable_count = 0
    start_time = time.time()
    
    while time.time() - start_time < timeout:
        elementos = driver.find_elements(By.CLASS_NAME, "flex.flex-col.grow.min-w-0")
        current_count = len(elementos)
        print(f"\rElementos encontrados: {current_count}", end="")
        
        if current_count == last_count:
            stable_count += 1
            if stable_count >= 3:
                print(f"\nCarregamento estabilizou com {current_count} elementos")
                return current_count
        else:
            stable_count = 0
            last_count = current_count
            start_time = time.time()
        
        time.sleep(1)
    
    print(f"\nTimeout atingido. Último número de elementos: {last_count}")
    return last_count

def scroll_page(driver):
    try:
        last_height = driver.execute_script("return document.body.scrollHeight")
        current_position = 0
        scroll_step = 300
        
        while current_position < last_height:
            current_position = min(current_position + scroll_step, last_height)
            driver.execute_script(f"window.scrollTo(0, {current_position});")
            time.sleep(random.uniform(0.2, 0.35))
            
            if random.random() < 0.2:
                add_random_actions(driver)
            
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height > last_height:
                last_height = new_height
            
            if current_position >= last_height:
                print("\nFim da página atingido. Aguardando carregamento final...")
                time.sleep(2)
                return verificar_carregamento_completo(driver)
        
        return verificar_carregamento_completo(driver)
        
    except Exception as e:
        print(f"Erro durante a rolagem: {e}")
        return 0

def extrair_dados_anuncio(anuncio_soup):
    dados = {}
    
    try:
        # Extrai localização
        localizacao_element = anuncio_soup.find("h2", {"data-cy": "rp-cardProperty-location-txt"})
        if localizacao_element:
            dados['Localidade'] = localizacao_element.text.strip().replace("Apartamento para comprar em", "").strip()
            
        # Extrai endereço
        endereco_element = anuncio_soup.find("p", {"data-cy": "rp-cardProperty-street-txt"})
        if endereco_element:
            dados['Endereco'] = endereco_element.text.strip()
            
        # Extrai área (m²)
        area_element = anuncio_soup.find("li", attrs={"data-cy": "rp-cardProperty-propertyArea-txt"})
        if area_element:
            area_text = area_element.text.strip()
            match = re.search(r'(\d+(?:\.|,)?\d*)\s*m²', area_text)
            if match:
                area_str = match.group(1).replace('.', '').replace(',', '.')
                dados['M2'] = float(area_str)
        
        # Método alternativo para encontrar área
        if 'M2' not in dados:
            for elemento in anuncio_soup.find_all(['li', 'span', 'h3']):
                if elemento.text and 'm²' in elemento.text:
                    match = re.search(r'(\d+(?:\.|,)?\d*)\s*m²', elemento.text)
                    if match:
                        area_str = match.group(1).replace('.', '').replace(',', '.')
                        dados['M2'] = float(area_str)
                        break

        # Extrai características (quartos, banheiros, vagas)
        caracteristicas = anuncio_soup.find_all("li", class_="flex row items-center gap-0-5")
        for item in caracteristicas:
            texto = item.text.strip()
            if item.get('data-cy') == 'rp-cardProperty-bedroomQuantity-txt':
                dados['Quartos'] = int(re.search(r'(\d+)', texto).group(1))
            elif item.get('data-cy') == 'rp-cardProperty-bathroomQuantity-txt':
                dados['Banheiros'] = int(re.search(r'(\d+)', texto).group(1))
            elif item.get('data-cy') == 'rp-cardProperty-parkingSpacesQuantity-txt':
                dados['Vagas'] = int(re.search(r'(\d+)', texto).group(1))
                
        # Extrai preço
        preco = None
        
        # Primeira tentativa: classe específica
        preco_element = anuncio_soup.find("p", class_="text-2-25 text-feedback-success-110 font-semibold")
        if not preco_element:
            preco_element = anuncio_soup.find("p", class_="text-2-25 text-neutral-120 font-semibold")
        
        if preco_element:
            preco_texto = preco_element.text.strip()
            preco_match = re.search(r'R\$\s*([\d.,]+)', preco_texto)
            if preco_match:
                preco = float(preco_match.group(1).replace('.', '').replace(',', '.'))
        
        # Segunda tentativa: procura por qualquer elemento com padrão de preço
        if not preco:
            for elemento in anuncio_soup.find_all(['p', 'span', 'div']):
                texto = elemento.text.strip()
                if 'R$' in texto and len(texto) < 50:
                    preco_match = re.search(r'R\$\s*([\d.,]+)', texto)
                    if preco_match:
                        preco = float(preco_match.group(1).replace('.', '').replace(',', '.'))
                        break
        
        if preco:
            dados['Preco'] = preco
        else:
            print("\nNão foi possível encontrar o preço do imóvel")
            return None
            
        # Extrai taxas (condomínio e IPTU)
        taxas_element = anuncio_soup.find("p", class_="text-1-75 text-neutral-110")
        if taxas_element:
            taxas_texto = taxas_element.text.strip()
            cond_match = re.search(r'Cond\.\s*R\$\s*([\d.,]+)', taxas_texto)
            iptu_match = re.search(r'IPTU\s*R\$\s*([\d.,]+)', taxas_texto)
            
            if cond_match:
                dados['Condominio'] = float(cond_match.group(1).replace('.', '').replace(',', '.'))
            if iptu_match:
                dados['IPTU'] = float(iptu_match.group(1).replace('.', '').replace(',', '.'))
        
        # Calcula preço por m²
        if 'Preco' in dados and 'M2' in dados and dados['M2'] > 0:
            dados['R$/M2'] = dados['Preco'] / dados['M2']
            
    except Exception as e:
        print(f"\nErro ao extrair dados do anúncio: {e}")
        return None
        
    return dados

def extrair_dados_pagina(url, max_paginas=10):
    driver = None
    data_list = []
    pagina_atual = 1
    
    try:
        driver = configure_driver()
        if not driver:
            print("Falha ao criar o driver. Verifique se o Chrome está instalado.")
            return None
            
        print("Driver configurado com sucesso!")
        driver.get(url)
        time.sleep(5)  # Aumentar tempo de espera inicial
        
        while pagina_atual <= max_paginas:
            try:
                # Verificar se o driver ainda está ativo
                driver.current_url
            except Exception as e:
                print(f"\nNavegador foi fechado ou perdeu conexão: {e}")
                print("Salvando dados coletados até agora...")
                return salvar_dados(data_list)
                
            clear_output(wait=True)
            print(f"\nProcessando página {pagina_atual}")
            print("Para parar: feche o navegador")
            
            try:
                # Aguardar carregamento da página com timeout maior
                WebDriverWait(driver, 20).until(
                    EC.presence_of_element_located((By.CLASS_NAME, "flex.flex-col.grow.min-w-0"))
                )
                
                num_elementos = scroll_page(driver)
                if num_elementos == 0:
                    print("Nenhum elemento encontrado após scroll")
                    break
                    
                print(f"\nIniciando extração de {num_elementos} elementos...")
                
                elementos = driver.find_elements(By.CLASS_NAME, "flex.flex-col.grow.min-w-0")
                for idx, elemento in enumerate(elementos, 1):
                    try:
                        print(f"\rProcessando elemento {idx}/{num_elementos}", end="")
                        html_content = elemento.get_attribute('outerHTML')
                        soup = BeautifulSoup(html_content, 'html.parser')
                        dados = extrair_dados_anuncio(soup)
                        
                        if dados:
                            data_list.append(dados)
                    except Exception as e:
                        print(f"\nErro ao processar elemento {idx}: {e}")
                        continue
                
                print(f"\nTotal de dados coletados até agora: {len(data_list)}")
                
                # Tentar encontrar botão de próxima página com múltiplos seletores
                next_button = None
                try:
                    # Primeira tentativa: seletor original
                    next_button = driver.find_element(By.CSS_SELECTOR, 'button[data-testid="next-page"]')
                except:
                    try:
                        # Segunda tentativa: seletor alternativo
                        next_button = driver.find_element(By.CSS_SELECTOR, 'button[aria-label*="próxima"]')
                    except:
                        try:
                            # Terceira tentativa: procurar por texto
                            next_button = driver.find_element(By.XPATH, "//button[contains(text(), 'Próxima') or contains(text(), 'Next')]")
                        except:
                            pass
                
                if next_button:
                    if not (next_button.is_enabled() and "disabled" not in next_button.get_attribute("class")):
                        print("\nNão há mais páginas para processar")
                        break
                        
                    driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", next_button)
                    time.sleep(random.uniform(1, 2))
                    next_button.click()
                    time.sleep(random.uniform(4, 6))
                    pagina_atual += 1
                else:
                    print("\nBotão de próxima página não encontrado. Finalizando extração.")
                    break
                    
            except Exception as e:
                print(f"\nErro ao processar página {pagina_atual}: {e}")
                # Tentar continuar com a próxima página se possível
                if pagina_atual < max_paginas:
                    print("Tentando continuar com a próxima página...")
                    pagina_atual += 1
                    time.sleep(5)
                    continue
                else:
                    break
    
    except Exception as e:
        print(f"\nErro durante a extração: {e}")
    
    finally:
        try:
            if driver:
                print("\nFechando navegador...")
                driver.quit()
        except Exception as e:
            print(f"Erro ao fechar navegador: {e}")
    
    return salvar_dados(data_list)

def salvar_dados(data_list):
    if data_list:
        df = pd.DataFrame(data_list)
        timestamp = time.strftime("%Y%m%d-%H%M%S")
        filename = f'dados_parciais_{timestamp}.csv'
        df.to_csv(filename, index=False)
        print(f"\nDados salvos em: {filename}")
        return df
    return None

def calcular_estatisticas(df):
    if 'R$/M2' in df.columns:
        media_aritmetica = df['R$/M2'].mean()
        media_ponderada = np.average(df['R$/M2'], weights=df['M2'])
        mediana = df['R$/M2'].median()
        moda = df['R$/M2'].mode()[0] if not df['R$/M2'].mode().empty else np.nan
        coef_var = df['R$/M2'].std() / df['R$/M2'].mean()
        total_linhas = len(df)

        print(f"Média Aritmética: {media_aritmetica:.2f}".replace('.', ','))
        print(f"Média Ponderada: {media_ponderada:.2f}".replace('.', ','))
        print(f"Mediana: {mediana:.2f}".replace('.', ','))
        print(f"Moda: {moda:.2f}".replace('.', ','))
        print(f"Coeficiente de Variação: {coef_var:.4f}".replace('.', ','))
        print(f"Total de Linhas: {total_linhas}")

def graficos(df, title_prefix=""):
    if not PLOTTING_AVAILABLE:
        print("Matplotlib não está disponível. Pulando geração de gráficos.")
        return
        
    if 'R$/M2' in df.columns:
        try:
            plt.figure(figsize=(10, 6))
            plt.hist(df['R$/M2'], bins=30, color='red', edgecolor='black', alpha=0.7)
            plt.title(f'{title_prefix}Histograma de Preço por M²')
            plt.xlabel('Preço por M²')
            plt.ylabel('Frequência')
            plt.grid(True)
            plt.show()
            
            plt.figure(figsize=(10, 6))
            plt.boxplot(df['R$/M2'], vert=False, showmeans=True)
            plt.title(f'{title_prefix}Boxplot de Preço por M²')
            plt.xlabel('Preço por M²')
            plt.grid(True)
            plt.show()
        except Exception as e:
            print(f"Erro ao gerar gráficos: {e}")

def remover_outliers_iqr(df, fator=1.5):
    if 'R$/M2' in df.columns:
        Q1 = df['R$/M2'].quantile(0.25)
        Q3 = df['R$/M2'].quantile(0.75)
        IQR = Q3 - Q1
        limite_inferior = Q1 - fator * IQR
        limite_superior = Q3 + fator * IQR
        return df[(df['R$/M2'] >= limite_inferior) & (df['R$/M2'] <= limite_superior)]
    return df

def remover_outliers_zscore(df, threshold=3):
    if 'R$/M2' in df.columns:
        z_scores = np.abs(zscore(df['R$/M2']))
        return df[z_scores < threshold]
    return df

def remover_outliers_iterativo(df, threshold=3, fator=1.5, max_iter=10):
    for _ in range(max_iter):
        df_old = df.copy()
        df = remover_outliers_iqr(df, fator)
        df = remover_outliers_zscore(df, threshold)
        if len(df) == len(df_old):
            break
    return df

def analisar_site(url_inicial):
    try:
        print("Iniciando extração de dados...")
        print("Para parar a extração, feche o navegador")
        
        df = extrair_dados_pagina(url_inicial)
        if df is not None and not df.empty:
            print("\nEstatísticas antes da remoção de outliers:")
            calcular_estatisticas(df)
            graficos(df, title_prefix="Antes de remover outliers - ")
            
            df_cleaned = remover_outliers_iterativo(df)
            
            print("\nEstatísticas após a remoção de outliers:")
            calcular_estatisticas(df_cleaned)
            graficos(df_cleaned, title_prefix="Após remoção iterativa de outliers - ")
            
            print("\nDataFrame após remoção iterativa de outliers:")
            display(df_cleaned)
            
            timestamp = time.strftime("%Y%m%d-%H%M%S")
            filename = f'dados_final_{timestamp}.csv'
            df_cleaned.to_csv(filename, index=False)
            print(f"\nDados finais salvos em: {filename}")
            
            return df_cleaned
    except Exception as e:
        print(f"Erro na análise: {e}")
    
    return None

if __name__ == "__main__":
    url_inicial = input("Digite a URL para análise: ")
    df_resultado = analisar_site(url_inicial)

# 🔧 Solução para Erro do ChromeDriver

## Problema Identificado
O erro `GetHandleVerifier` que você está enfrentando é causado por incompatibilidade entre a versão do ChromeDriver e a versão do Chrome instalada no seu sistema.

## Soluções Implementadas

### 1. **Configuração Robusta do Driver**
- ✅ Múltiplas tentativas com diferentes versões do ChromeDriver
- ✅ Verificação automática se o Chrome está instalado
- ✅ Atualização automática do ChromeDriver
- ✅ Configurações adicionais para evitar crashes

### 2. **Melhor Tratamento de Erros**
- ✅ Timeouts aumentados para conexões lentas
- ✅ Múltiplos seletores para encontrar botões de navegação
- ✅ Recuperação automática de falhas de conexão
- ✅ Salvamento automático de dados em caso de erro

## Como Usar

1. **Execute a célula de código atualizada** - ela tentará automaticamente diferentes versões do ChromeDriver
2. **Se ainda houver problemas**, execute o comando abaixo para atualizar manualmente:

```python
# Atualizar ChromeDriver manualmente
import subprocess
import sys
subprocess.run([sys.executable, "-m", "pip", "install", "--upgrade", "undetected-chromedriver"])
```

3. **Reinicie o kernel** do Jupyter e execute novamente

## Alternativas se o Problema Persistir

### Opção 1: Usar Firefox
```python
from selenium import webdriver
from selenium.webdriver.firefox.options import Options

def configure_firefox_driver():
    options = Options()
    options.add_argument('--headless')  # Opcional: executar sem interface
    driver = webdriver.Firefox(options=options)
    return driver
```

### Opção 2: Usar Edge
```python
from selenium import webdriver
from selenium.webdriver.edge.options import Options

def configure_edge_driver():
    options = Options()
    driver = webdriver.Edge(options=options)
    return driver
```

## Verificação do Sistema
Execute o código abaixo para verificar se tudo está funcionando:

```python
# Verificar Chrome
verificar_chrome_instalado()

# Tentar criar driver
driver = configure_driver()
if driver:
    print("✅ Driver configurado com sucesso!")
    driver.quit()
else:
    print("❌ Problema na configuração do driver")
```


In [2]:
# Teste de configuração do driver
print("🔍 Verificando configuração do sistema...")

# Verificar Chrome
chrome_ok = verificar_chrome_instalado()

if chrome_ok:
    print("\n🚀 Tentando configurar o driver...")
    driver = configure_driver()
    
    if driver:
        print("✅ Driver configurado com sucesso!")
        print("🎉 Sistema pronto para uso!")
        
        # Teste rápido
        try:
            driver.get("https://www.google.com")
            print("✅ Teste de navegação bem-sucedido!")
            driver.quit()
        except Exception as e:
            print(f"❌ Erro no teste de navegação: {e}")
    else:
        print("❌ Falha na configuração do driver")
        print("💡 Tente atualizar o ChromeDriver manualmente:")
        print("   pip install --upgrade undetected-chromedriver")
else:
    print("❌ Chrome não encontrado. Instale o Google Chrome primeiro.")


🔍 Verificando configuração do sistema...
Chrome encontrado em: C:\Program Files\Google\Chrome\Application\chrome.exe

🚀 Tentando configurar o driver...
Chrome encontrado em: C:\Program Files\Google\Chrome\Application\chrome.exe
Tentando versão automática...
Falha com versão automática: Message: invalid argument: cannot parse capability: goog:chromeOptions
from invalid argument: unreco...
Tentando versão 120...
Falha com versão 120: you cannot reuse the ChromeOptions object...
Tentando versão 119...
Falha com versão 119: you cannot reuse the ChromeOptions object...
Tentando versão 118...
Falha com versão 118: you cannot reuse the ChromeOptions object...
Tentando versão padrão...
Falha com versão padrão: you cannot reuse the ChromeOptions object...
Não foi possível criar o driver em nenhuma tentativa
Tentando atualizar ChromeDriver...
Tentando atualizar ChromeDriver...
ChromeDriver atualizado com sucesso!
Tente executar o código novamente após a atualização.
❌ Falha na configuração do dr